In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
### YOUR CODE HERE
from tensorflow.keras import regularizers
###
import tensorflow.keras.utils as ku 
import numpy as np
import keras
from keras import layers
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [9]:
tokenizer = Tokenizer()
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt \
    -O /tmp/sonnets.txt
data = open('/tmp/sonnets.txt').read()

corpus = data.lower().split("\n")


tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

--2021-02-10 17:08:26--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.217.128, 173.194.215.128, 172.217.193.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.217.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘/tmp/sonnets.txt’

/tmp/sonnets.txt    100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2021-02-10 17:08:26 (71.8 MB/s) - ‘/tmp/sonnets.txt’ saved [93578/93578]



In [10]:
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

In [11]:

model = keras.models.Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(layers.Bidirectional(layers.LSTM(512, return_sequences=True)))
model.add(layers.LSTM(256))
model.add(layers.Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics='accuracy')



In [12]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 10, 100)           321100    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 10, 512)           731136    
_________________________________________________________________
lstm_5 (LSTM)                (None, 128)               328192    
_________________________________________________________________
dense_2 (Dense)              (None, 3211)              414219    
Total params: 1,794,647
Trainable params: 1,794,647
Non-trainable params: 0
_________________________________________________________________


In [13]:
address="./kaggle/working/weights-improvement.hdf5"
stop = EarlyStopping(monitor = 'val_loss', min_delta = 0, 
                             patience = 5, verbose = 1, mode = 'auto')
save = ModelCheckpoint(address, monitor = 'val_loss', 
                               verbose = 0, save_best_only = True)
callbacks = [stop, save]

history = model.fit(predictors, label, validation_split=0.20, epochs=50, verbose=1, callbacks = callbacks)

Epoch 1/50
387/387 [==============================] - 5s 14ms/step - loss: 6.9270 - accuracy: 0.0226 - val_loss: 6.8114 - val_accuracy: 0.0184
Epoch 2/50
387/387 [==============================] - 4s 10ms/step - loss: 6.4593 - accuracy: 0.0340 - val_loss: 6.9325 - val_accuracy: 0.0343
Epoch 3/50
387/387 [==============================] - 4s 11ms/step - loss: 6.2966 - accuracy: 0.0362 - val_loss: 7.0280 - val_accuracy: 0.0236
Epoch 4/50
387/387 [==============================] - 4s 10ms/step - loss: 6.1748 - accuracy: 0.0406 - val_loss: 7.0752 - val_accuracy: 0.0255
Epoch 5/50
387/387 [==============================] - 4s 10ms/step - loss: 6.0520 - accuracy: 0.0457 - val_loss: 7.0884 - val_accuracy: 0.0268
Epoch 6/50
387/387 [==============================] - 4s 11ms/step - loss: 5.9131 - accuracy: 0.0514 - val_loss: 7.1423 - val_accuracy: 0.0281
Epoch 00006: early stopping


The validation loss kept increasing so after patience=5 the training stopped.